# seq to seq and attention

In [41]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

In [42]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [43]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
SOS_token = 0
EOS_token = 1

In [46]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            # if not SOS or EOS, then the index 
            # will start from 2
            self.word2index[word] = self.n_words
            # The first actual word with counting
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

In [47]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [48]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    lines = open("./data/english-lang-trans/%s-%s.txt" % (lang1, lang2),\
                encoding="utf-8").read().strip().split("\n")
    
    pairs = [[normalizeString(s) for s in l.split("\t")] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    return input_lang, output_lang, pairs 

In [49]:
# trim the data set to only relatively short and simple sentences.

MAX_LENGTH=10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [50]:
# prepare dataset

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s senence pairs" % len(pairs))
    print("Counting words")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        
    print("Counted words: ")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [51]:
input_lang, output_lang, pairs = prepareData("eng", "fra", True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 senence pairs
Counting words
Counted words: 
fra 4601
eng 2991
['je suis sur de l avoir vu auparavant', 'i m sure i ve seen him before']


## seq2seq working flow
![seq2seq working flow](https://pytorch.org/tutorials/_images/encoder-network.png)

In [52]:
class EncoderRNN(nn.Module):
    def __init__(self,input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)
        
    
    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

![seq2seq encoding - decoding](https://pytorch.org/tutorials/_images/decoder-network.png)

**Understanding squeeze(-1) and Detach in Decoder Input Processing (Seq2seq with Teacher Forcing):**

In a seq2seq model with teacher forcing, `squeeze(-1)` and `detach` are used together during decoder input processing to ensure proper shape compatibility and memory optimization. Here's a detailed breakdown:

**1. Context and Teacher Forcing:**

- Seq2seq models translate between sequences (e.g., text-to-text, speech-to-text).
- Teacher forcing is a training technique where the model receives the correct target sequence as input at each decoding step, along with previously generated elements.

**2. Decoder Input Processing:**

   **a. Target Sequence Reshaping (unsqueeze(1))**

     - The target sequence (ground truth) typically has a shape `(batch_size, target_length)`.
     - For teacher forcing, the decoder needs input one element at a time during each decoding step.
     - To achieve this, `unsqueeze(1)` is used during training:

       ```python
       # Example target sequence
       target_sequence = torch.randint(1, 10, (batch_size, target_length))

       # Teacher forcing input for first decoding step
       teacher_forcing_input = target_sequence[:, 0].unsqueeze(1)
       ```

     - Explanation:
       - `target_sequence[:, 0]` extracts the first element (index 0) from each sequence in the batch, resulting in a tensor of shape `(batch_size,)`.
       - `unsqueeze(1)` inserts a new dimension of size 1 at dimension 1 (the column dimension). This creates a tensor with shape `(batch_size, 1)`, aligning with the expected decoder input format at the first step.

   **b. Decoder Output Reshaping and Detachment (squeeze(-1).detach())**

     - After each decoding step, the decoder generates an output.
     - We need to compare the decoder output with the corresponding element in the target sequence for calculating the loss during training.
     - However, the decoder output might have a shape `(batch_size, target_length, hidden_size)`:

       - `batch_size`: Number of samples in the batch.
       - `target_length`: Length of the target sequence.
       - `hidden_size`: Model's internal hidden state dimension (representing extracted features).

     ```python
     # Example decoder output after a decoding step
     decoder_output = model(decoder_input)  # Model processes decoder input

     # Process decoder output for loss calculation
     processed_decoder_output = decoder_output.squeeze(-1).detach()
     ```

     - Explanation:
       - `squeeze(-1)` removes the dimension of size 1 at the last dimension (dimension -1). This ensures the decoder output has the same shape `(batch_size, target_length)` as the target sequence, allowing for element-wise comparison during loss calculation. Essentially, it extracts the meaningful content from the last dimension (`hidden_size`) by combining it with the previous dimensions (`batch_size` and `target_length`).
       - `.detach()` detaches the processed decoder output from the computational graph. Since we're using the ground truth for teacher forcing and not backpropagating through the decoder output in this step, detaching saves memory and avoids unnecessary computations.

**3. Key Points:**

- `unsqueeze(1)` prepares the target sequence for teacher forcing by creating an input with the correct shape for the first decoding step.
- `squeeze(-1).detach()` processes the decoder output by:
   - Removing the unnecessary `hidden_size` dimension for element-wise comparison.
   - Detaching the output from the computational graph for teacher forcing efficiency.

**In essence, these operations ensure that the decoder receives the appropriate teacher forcing input and that the decoder output is properly shaped for loss calculation during training.**

In [53]:
class DecoderRNN(nn.Module):
    def __init__(self,hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long,\
                                   device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        
        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input,
                                                               decoder_hidden)
            decoder_outputs.append(decoder_output)
            
            if target_tensor is not None:
                # Teacher forcing, feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # teacher forcing
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach() # detach from hist as input
                
        decoder_outputs = torch.cat(decoer_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        # return `None` for consistency in the training loop
        return decoder_outputs, decoder_hidden, None
    
    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden
        

![](https://i.imgur.com/1152PYf.png)  
![](https://pytorch.org/tutorials/_images/attention-decoder-network.png)


In [54]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

In [55]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [56]:
# preparing taining data

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(" ")]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData("eng", "fra", True)
    
    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    
    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids
        
        
    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))
    
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, 
                                  batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [57]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [58]:
# This is a helper function to print time elapsed and estimated time remaining given the current time and progress %.

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [59]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

> Change the backend to TkAgg, QtAgg, or WXAgg

In [71]:
import matplotlib.pyplot as plt
plt.switch_backend('QtAgg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [61]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [62]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [63]:
# actual train, evaluation

hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 senence pairs
Counting words
Counted words: 
fra 4601
eng 2991
0m 26s (- 6m 31s) (5 6%) 1.5495
0m 50s (- 5m 56s) (10 12%) 0.6837
1m 16s (- 5m 29s) (15 18%) 0.3479
1m 41s (- 5m 3s) (20 25%) 0.1897
2m 7s (- 4m 40s) (25 31%) 0.1159
2m 33s (- 4m 15s) (30 37%) 0.0799
2m 59s (- 3m 50s) (35 43%) 0.0606
3m 25s (- 3m 25s) (40 50%) 0.0503
3m 51s (- 3m 0s) (45 56%) 0.0437
4m 18s (- 2m 35s) (50 62%) 0.0387
4m 44s (- 2m 9s) (55 68%) 0.0367
5m 10s (- 1m 43s) (60 75%) 0.0336
5m 36s (- 1m 17s) (65 81%) 0.0327
6m 2s (- 0m 51s) (70 87%) 0.0314
6m 28s (- 0m 25s) (75 93%) 0.0293
6m 54s (- 0m 0s) (80 100%) 0.0288


In [64]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> elle rassemble du materiel pour un livre
= she is collecting material for a book
< she s collecting material for a book <EOS>

> tu n es pas ma mere
= you re not my mother
< you aren t my mother <EOS>

> je m en rends compte
= i m fully aware of that
< i m fully aware of that <EOS>

> je ne suis vraiment pas suppose faire ca
= i m really not supposed to do this
< i m really not supposed to do this <EOS>

> vous etes plante
= you re stuck
< you re stuck <EOS>

> elle est plus une relation qu une amie
= she is more an acquaintance than a friend
< she is more an acquaintance than a friend <EOS>

> vous etes degoutants
= you re disgusting
< you re disgusting <EOS>

> ils sont dangereux
= they re dangerous
< they re dangerous <EOS>

> ils sont encore des enfants
= they are still children
< they are still children back <EOS>

> vous etes tres marrantes
= you re very funny
< you re very funny man <EOS>



In [72]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father <EOS>


/tmp/ipykernel_29049/1690937169.py:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
/tmp/ipykernel_29049/1690937169.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


input = je suis trop fatigue pour conduire
output = i m too tired to drive <EOS>
input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>
